In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/EMRES'

/content/drive/MyDrive/EMRES


In [ ]:
import torch
import torch.nn as nn
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 34.5 MB/s eta 0:00:00


In [ ]:
music_feature_norm = torch.load('music_feature_norm.pt')
music_emotion_norm = torch.load('music_emotion_norm.pt')

input 선택

원래는 사용자의 현재 감정유사도를 기반으로 해야하지만 argMax가 없으니 했다고 가정

In [ ]:
first_vector_feature = music_feature_norm[0] #shape이 (124,) 입력으로 들어갈 것
first_vector_emotion = music_emotion_norm[0] #shape이 (2,) 입력으로 들어갈 것

In [ ]:
first_vector_feature = first_vector_feature.unsqueeze(0).unsqueeze(1)
first_vector_emotion = first_vector_emotion.unsqueeze(0).unsqueeze(1)

In [ ]:
music_feature_norm_add = music_feature_norm.unsqueeze(2)
music_emotion_norm_add = music_emotion_norm.unsqueeze(2)

In [ ]:
print(first_vector_feature.shape)
print(first_vector_emotion.shape)
print(music_feature_norm_add.shape)
print(music_emotion_norm_add.shape)

torch.Size([1, 1, 124])
torch.Size([1, 1, 2])
torch.Size([34, 124, 1])
torch.Size([34, 2, 1])


argmax가 안되니까
모델 입력으로 이미 사용자가 고른 노래의 feauture(normalized)와 emotion(normalized)가 들어옴

normalize는 cos유사도를 해야하는데, 크기구하는 메서드가 없기 때문에 크기를 1로 만들어주려는 것


---

노래 번호도 원래 인덱싱으로 뽑아야하는데 일단은 34곡에 대해서만 0부터 시작하는 걸로 수정

In [ ]:
class EMRES(nn.Module):
    def __init__(self, music_emotion, music_feature):
        super().__init__()
        self.n = 16

        self.alpha = nn.Parameter(torch.ones(1,)*0.2, requires_grad=False)
        self.beta = nn.Parameter(torch.ones(1,)*0.8, requires_grad=False)


        self.music_emotion = music_emotion.squeeze(2)
        self.music_feature = music_feature.squeeze(2)

        self.similarity_emotion = nn.Linear(2, 34, bias=False)
        self.similarity_emotion.weight.data = self.music_emotion
        self.similarity_emotion.weight.requires_grad = False


        self.similarity_feature = nn.Linear(124, 34, bias=False)
        self.similarity_feature.weight.data = self.music_feature
        self.similarity_feature.weight.requires_grad = False

    def forward(self, user_com):
        usermusic_emotion = user_com[:,:,:2]
        usermusic_feature = user_com[:,:,2:]
        #  아래 2줄로 인해 multi input error 발생
        cos_emotion = self.similarity_emotion(usermusic_emotion)
        cos_feature = self.similarity_feature(usermusic_feature)

        out = self.alpha * cos_emotion + self.beta * cos_feature
        print(out.shape)
        return out

In [ ]:
usermusic_emotion = torch.rand(1,1,2)
usermusic_feature = torch.rand(1,1,124)
user_com = torch.cat([usermusic_emotion,usermusic_feature],2)
emres = EMRES(music_emotion_norm_add, music_feature_norm_add)
emres.eval()
torch.onnx.export(emres,user_com, "EMRES_ONNX.onnx")

torch.Size([1, 1, 34])


사용자 input - 예시 : [(143, 1, 3), (381, 1, 2), (388, 1, 4), (389, 1, 4)] -

텐서로 만들자면 (1,데이터 개수, 곡의 제목)으로 하고 요소값을 감정으로 하면 될듯(1은 배치용)

리스트안에 여러 요소가 있고 각 요소는 곡 제목, 좋아요, 노래들을 때의 감정

노래 input

music_vectors (1,곡의 제목, 벡터)

music_emotion(1, 곡의 제목, 벡터)